In [1]:
import numpy as np
import fiftyone as fo
import fiftyone.zoo as foz
import fiftyone.brain as fob
from fiftyone import ViewField as F

In [2]:
import pandas as pd

def count_dict_to_df(dataset_id: str, count_dict: dict, columns: list) -> pd.DataFrame:
    list_to_df = []
    for k in count_dict.keys():
        row_df = [dataset_id, k, count_dict[k]]
        list_to_df.append(row_df)
        
    df = pd.DataFrame(data=list_to_df, columns=columns)
    return df        

In [3]:
print(fo.list_datasets())

['CNNW', 'FOXNEWSW', 'MSNBCW', 'hodost-lv', 'news7-lv']


# Dataset visualization

In [14]:
dataset = fo.load_dataset("FOXNEWSW")

In [113]:
# Used for dev.
#dataset.delete_sample_field("embeddings")
#dataset.delete()

In [102]:
# Used for dev.
# In case we want to delete the associated results (used during development)
#dataset = fo.load_dataset("CNNW")
#print(dataset)
#dataset.delete_sample_field("yolo-resnetv1-fcg_average_vote")
#print(dataset)

In [15]:
# If we don't specify the "eval_key=eval" it doesn't compute the ious
dataset.evaluate_detections("yolo-resnetv1-fcg_average_vote", "ground_truth", eval_key="eval")
#dataset.evaluate_detections("yolo-resnetv1-fcg_average_vote", "ground_truth")

#fo.utils.iou.compute_ious(dataset["yolo-resnetv1-fcg_average_vote"], "ground_truth")
#fo.utils.iou.compute_max_ious(dataset, "ground_truth", iou_attr="max_iou", classwise=True)

Evaluating detections...
 100% |███████████████| 1674/1674 [7.3s elapsed, 0s remaining, 245.9 samples/s]       


In [16]:
# In case we want to make a view to reduce the amount of data to be shown
view = dataset.match(F("year").is_in(["2012"]))
view = dataset.filter_labels("yolo-resnetv1-fcg_average_vote", F("eval_iou") > 0.1)
#ep = view.to_evaluation_patches("eval")
#ep.match(F("iou") > 0.9)
#view = view.filter_labels("yolo-resnetv1-fcg_average_vote", F("iou") > 0.1)
              
# Open session in a tab
session = fo.launch_app(view, auto=False)
session.open_tab()
# Open session in jupyer output
#session = fo.launch_app(dataset)

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [ ]:
# Filter the dataset by label. The problem is that the embeddings are not aware of this filtering,
# so they can't be matched :(
#from fiftyone import ViewField as F

#dataset = fo.load_dataset("CNNW").take(10)

#view = dataset.filter_labels(
#    "yolo-resnetv1-fcg_average_vote", F("label").is_in(["Donald_Trump", "Barack_Obama"])
#)

# Get detections by ID (not label) and exclude them from the dataset
#ids = [
#    dataset.last()['yolo-resnetv1-fcg_average_vote'].detections[0].id,
#]
# view = dataset.exclude_labels(ids=ids)
# print(ids)

In [6]:
# Load the previously produced embeddings
import numpy as np

#dataset = fo.load_dataset("CNNW")
embeddings = {i: e for i, e in zip(*dataset.values(["id", "embeddings"]))}

results = fob.compute_visualization(
    dataset, patches_field="yolo-resnetv1-fcg_average_vote", embeddings=embeddings, num_dims=2
)

Generating visualization...
UMAP( verbose=True)
Mon Jun 27 15:14:07 2022 Construct fuzzy simplicial set
Mon Jun 27 15:14:07 2022 Finding Nearest Neighbors
Mon Jun 27 15:14:07 2022 Building RP forest with 21 trees
Mon Jun 27 15:14:08 2022 NN descent for 17 iterations
	 1  /  17
	 2  /  17
	 3  /  17
	 4  /  17
	 5  /  17
	Stopping threshold met -- exiting after 5 iterations
Mon Jun 27 15:14:24 2022 Finished Nearest Neighbor Search
Mon Jun 27 15:14:27 2022 Construct embedding


Epochs completed:   0%|            0/200 [00:00]

Mon Jun 27 15:15:18 2022 Finished embedding


In [ ]:
# Example for computing the embeddings with a model from the zoo
#dataset = fo.load_dataset("CNNW").take(10)
#dataset = fo.load_dataset("hodost-lv")
#dataset_patches = dataset.to_patches("ground_truth")
#dataset_patches = dataset.to_patches("yolo-resnetv1-fcg_average_vote")
#foz.models.list_zoo_models()
#model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
#embs_test = dataset.compute_patch_embeddings(model, "yolo-resnetv1-fcg_average_vote")
# Compute visualization
#results = fob.compute_visualization(dataset, patches_field="yolo-resnetv1-fcg_average_vote", embeddings=embs_test, seed=51)

In [ ]:
print(dataset.get_classes("yolo-resnetv1-fcg_average_vote"))

In [7]:
# Which "classes" to plot
#classes_list = dataset.get_classes("yolo-resnetv1-fcg_average_vote")
from utils import PRIME_MINISTERS

classes_list = list(set(PRIME_MINISTERS)) # Get unique classes
classes_ignore = ['-1']
classes_to_plot = [c for c in classes_list if c not in classes_ignore]
# Generate scatterplot
plot = results.visualize(
    #labels="ground_truth.detections.label",
    #labels="ground_truth.label",
    #labels="yolo-resnetv1-fcg_average_vote.label",
    labels="yolo-resnetv1-fcg_average_vote.detections.label",
    labels_title="ID",
    classes=classes_to_plot,
    axis_equal=True,
)
plot.show(height=512)

session.plots.attach(plot)

FigureWidget({
    'data': [{'customdata': array(['62b5572436ba2283d6363a65', '62b5572436ba2283d6363a68',
    …

# Evaluation

In [97]:
# Filter the dataset
from fiftyone import ViewField as F

# Only contains detections with confidence >= 0.75
#high_conf_view = predictions_view.filter_labels("yolo-resnetv1-fcg_average_vote", F("confidence") > 0.75)

In [15]:
# Resource: https://voxel51.com/docs/fiftyone/tutorials/evaluate_detections.html
# Evaluate the predictions in the `faster_rcnn` field of our `high_conf_view`
# with respect to the objects in the `ground_truth` field
dataset = fo.load_dataset("CNNW")

results = dataset.evaluate_detections(
    "yolo-resnetv1-fcg_average_vote",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)

Evaluating detections...
 100% |███████████████| 1083/1083 [5.2s elapsed, 0s remaining, 228.9 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1083/1083 [5.2s elapsed, 0s remaining, 232.3 samples/s]      


In [16]:
results.metrics()

{'accuracy': 0.3283533614810003,
 'precision': 0.3362154971732624,
 'recall': 0.9335180055401662,
 'fscore': 0.4943765281173594,
 'support': 1083}

In [99]:
# Get the 10 most common classes in the dataset
counts = dataset.count_values("ground_truth.detections.label")
classes_top10 = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
results.print_report(classes=classes_top10)

                  precision    recall  f1-score   support

Kellyanne_Conway       0.96      0.98      0.97        66
 Hillary_Clinton       0.61      0.96      0.74        48
 Jon_Huntsman_Jr       0.80      0.96      0.87        45
   Rick_Santorum       0.78      0.98      0.87        41
    Gary_Johnson       1.00      1.00      1.00        39
     Jim_Gilmore       0.95      0.95      0.95        38
       Rand_Paul       0.62      0.86      0.72        37
  Lincoln_Chafee       0.72      0.97      0.83        37
George_Zimmerman       0.78      0.89      0.83        36
  Lindsey_Graham       0.92      0.97      0.94        35

       micro avg       0.80      0.95      0.87       422
       macro avg       0.81      0.95      0.87       422
    weighted avg       0.82      0.95      0.88       422



In [100]:
print(results.mAP())

0.2743891904744837


In [19]:
plot = results.plot_pr_curves(classes=["Shinzo_Abe", "Yoshihide_Suga"])
plot.show()

FigureWidget({
    'data': [{'customdata': array([0.91272851, 0.909137  , 0.90479841, 0.90406826, 0.90289347, …

In [39]:
#sample = dataset.first()
#print(sample['yolo-resnetv1-fcg_average_vote'].detections[0])
#print(dataset.get_evaluation_info("eval"))

In [106]:
# Compute metadata so we can reference image height/width in our view
dataset.compute_metadata()

In [51]:
(96**2 / 84480) * 100

10.909090909090908

In [24]:
#
# Create an expression that will match objects whose bounding boxes have
# area less than 32^2 pixels
#
# Bounding box format is [top-left-x, top-left-y, width, height]
# with relative coordinates in [0, 1], so we multiply by image
# dimensions to get pixel area
#
bbox_area = (
    F("$metadata.width") * F("bounding_box")[2] *
    F("$metadata.height") * F("bounding_box")[3]
)
small_boxes = bbox_area < 32 ** 2
#medium_boxes = (32 ** 2 < bbox_area) & (bbox_area < 96 ** 2)
medium_boxes = (bbox_area >= 32 ** 2) & (bbox_area <= 96 ** 2)
large_boxes = bbox_area > 96 ** 2

In [59]:
#medium_boxes
sample = dataset.first()
print(sample.ground_truth.detections[0]['bounding_box'])

[0.4217329545454546, 0.21812499999999999, 0.14176136363636363, 0.27291666666666664]


In [25]:
# Create a view that contains only small GT and predicted boxes
small_boxes_eval_view = (
    dataset
    .filter_labels("ground_truth", small_boxes)
    .filter_labels("yolo-resnetv1-fcg_average_vote", small_boxes)
    .filter_labels("yolo-resnetv1-fcg_average_vote", F("label") != "-1")
)
medium_boxes_eval_view = (
    dataset
    .filter_labels("ground_truth", medium_boxes)
    .filter_labels("yolo-resnetv1-fcg_average_vote", medium_boxes)
    .filter_labels("yolo-resnetv1-fcg_average_vote", F("label") != "-1")    
)
large_boxes_eval_view = (
    dataset
    .filter_labels("ground_truth", large_boxes)
    .filter_labels("yolo-resnetv1-fcg_average_vote", large_boxes)
    .filter_labels("yolo-resnetv1-fcg_average_vote", F("label") != "-1")    
)

# Run evaluation
small_boxes_results = small_boxes_eval_view.evaluate_detections(
    "yolo-resnetv1-fcg_average_vote",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)
medium_boxes_results = medium_boxes_eval_view.evaluate_detections(
    "yolo-resnetv1-fcg_average_vote",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)
large_boxes_results = large_boxes_eval_view.evaluate_detections(
    "yolo-resnetv1-fcg_average_vote",
    gt_field="ground_truth",
    eval_key="eval",
    compute_mAP=True,
)


Evaluating detections...
 100% |███████████████| 1569/1569 [24.6s elapsed, 0s remaining, 72.5 samples/s]      
Performing IoU sweep...
 100% |███████████████| 1569/1569 [11.0s elapsed, 0s remaining, 156.1 samples/s]      
Evaluating detections...
 100% |███████████████| 6329/6329 [48.7s elapsed, 0s remaining, 148.5 samples/s]      
Performing IoU sweep...
 100% |███████████████| 6329/6329 [22.4s elapsed, 0s remaining, 308.5 samples/s]      
Evaluating detections...
 100% |█████████████████| 343/343 [2.3s elapsed, 0s remaining, 152.8 samples/s]      
Performing IoU sweep...
 100% |█████████████████| 343/343 [1.1s elapsed, 0s remaining, 311.0 samples/s]         


In [56]:
small_boxes_eval_view.bounds("ground_truth")

(<Detections: {
     'detections': BaseList([
         <Detection: {
             'id': '62aa9c95942ad79132de47e0',
             'attributes': BaseDict({}),
             'tags': BaseList([]),
             'label': 'Yukio_Edano',
             'bounding_box': BaseList([
                 0.16676136363636365,
                 0.49291666666666667,
                 0.06079545454545454,
                 0.115,
             ]),
             'mask': None,
             'confidence': None,
             'index': None,
             'label_id': 'Edano',
         }>,
         <Detection: {
             'id': '62aa9c95942ad79132de47e1',
             'attributes': BaseDict({}),
             'tags': BaseList([]),
             'label': 'Toru_Hashimoto',
             'bounding_box': BaseList([
                 0.6762784090909091,
                 0.5164583333333332,
                 0.07187500000000001,
                 0.12208333333333334,
             ]),
             'mask': None,
             'confide

In [26]:
# Get the 10 most common small object classes
small_counts = small_boxes_eval_view.count_values("ground_truth.detections.label")
medium_counts = medium_boxes_eval_view.count_values("ground_truth.detections.label")
large_counts = large_boxes_eval_view.count_values("ground_truth.detections.label")
classes_top10_small = sorted(small_counts, key=counts.get, reverse=True)[:10]
classes_top10_medium = sorted(medium_counts, key=counts.get, reverse=True)[:10]
classes_top10_large = sorted(medium_counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 small object classes
small_boxes_results.print_report(classes=classes_top10_small)
medium_boxes_results.print_report(classes=classes_top10_medium)
large_boxes_results.print_report(classes=classes_top10_large)

                  precision    recall  f1-score   support

      Shinzo_Abe       0.99      0.83      0.90       558
  Yoshihide_Suga       0.68      0.92      0.79       286
        Taro_Aso       0.95      0.53      0.68       313
   Fumio_Kishida       1.00      0.32      0.48       273
     Yukio_Edano       0.56      0.31      0.40       378
      Kazuo_Shii       0.66      0.67      0.67       372
   Katsuya_Okada       0.86      0.66      0.75       195
 Yuichiro_Tamaki       0.80      0.75      0.78       287
   Ichiro_Matsui       0.98      0.22      0.36       228
Natsuo_Yamaguchi       0.89      0.41      0.56       262

       micro avg       0.81      0.59      0.68      3152
       macro avg       0.84      0.56      0.64      3152
    weighted avg       0.83      0.59      0.66      3152

                  precision    recall  f1-score   support

      Shinzo_Abe       0.99      0.84      0.91      1315
  Yoshihide_Suga       0.94      0.99      0.96      1070
        Ta

In [27]:
# Default behavior: only show rows (GT) for the requested classes, but include
# "other" and "missing" columns that capture the rest of the predictions associated
# with the GT labels of interest
medium_boxes_results.plot_confusion_matrix(classes=classes_top10_medium)

FigureWidget({
    'data': [{'mode': 'markers',
              'opacity': 0.1,
              'type': 'scatter',…

In [28]:
plot = medium_boxes_results.plot_pr_curves(classes=["Shinzo_Abe", "Yoshihide_Suga"])
plot.show()

FigureWidget({
    'data': [{'customdata': array([0.91272851, 0.90504362, 0.90035601, 0.89917585, 0.8984457 , …

In [35]:
#session = fo.launch_app(dataset, auto=False)
session = fo.launch_app(dataset, auto=False)
session.open_tab()

Session launched. Run `session.show()` to open the App in a cell output.


<IPython.core.display.Javascript object>

In [37]:
session.view = large_boxes_eval_view.sort_by("eval_fp", reverse=True)

TypeError: sort_by() got an unexpected keyword argument 'auto'

# Analysis for figures

In [16]:
dataset_orig = fo.load_dataset("news7-lv")
dataset_orig.compute_metadata()
# Do "evaluate_detections" to compute iou to be able to threshold wrt iou for US data evaluation
# Important: do "classwise=False" to consider FNs
#dataset_orig.evaluate_detections("yolo-resnetv1-fcg_average_vote", "ground_truth", eval_key="eval", classwise=False)

# For NHK missing data
years_list = [str(i) for i in range(2013, 2022)]
view_analysis = dataset_orig.match(F("year").is_in(years_list))

# For US evaluation
# Filter the detections based on the IoU threshold
# So here was a funny bug that took me all the afternoon to find.
# If you filter a dataset to a view object, and you compute the evaluation over that view, 
# weird things happen. I realised this after filtering by "eval_iou", that in a further step
# it did not return anything when doing "evaluate_detections" some cells below.
# To bypass this bug, convert a view to a dataset by using "clone()" once filtered by iou.
#view_analysis = view_analysis.filter_labels("yolo-resnetv1-fcg_average_vote", F("eval_iou") > 0.001).clone()

# Generate different views depending on the bounding box sizes 
bbox_area = (
    F("$metadata.width") * F("bounding_box")[2] *
    F("$metadata.height") * F("bounding_box")[3]
)
# [very small, small, small-medium, medium, medium-large, large, very large]
# Average bbox for NHK = 78x78, HODO = 52x52. US dataset around 135 x 135.
# Smallest NHK = 3x3, HODO = 2x2. US = 35x35
# Largest NHK = 258x258, HODO = 174x174. US = 390x390

boxes_areas = list(map(int, list(np.asarray([8, 16, 32, 64, 96, 128, 156]) ** 2)))
boxes_filter_list = []

for i in range(len(boxes_areas)):
    if i == 0:
        # Skip 0, used for posterior plots
        boxes_filter = bbox_area <= boxes_areas[i]
    else:
        # Cases in the middle
        boxes_filter = (bbox_area > boxes_areas[i-1]) & (bbox_area <= boxes_areas[i])

    boxes_filter_list.append(boxes_filter)
        
# Last case
boxes_filter_list.append(bbox_area > boxes_areas[-1])


In [17]:
# Generate views that contains only the filtered bboxes depending on size
views_list = []

for box_filter in boxes_filter_list:
#for box_filter in [small_boxes, medium_boxes]:
    view_filtered = (
        view_analysis
        .filter_labels("ground_truth", box_filter)
        .filter_labels("yolo-resnetv1-fcg_average_vote", box_filter)
        .filter_labels("yolo-resnetv1-fcg_average_vote", F("label") != "-1")
    )
    views_list.append(view_filtered)

In [ ]:
# Run evaluation for the generated filtered views
results_list = []

for view_filtered in views_list:
    results_filtered = view_filtered.evaluate_detections(
        "yolo-resnetv1-fcg_average_vote",
        gt_field="ground_truth",
        eval_key="eval",
        compute_mAP=True,
        #iou_threshs=[0.4, 0.45, 0.5, 0.55, 0.6],  # For US evaluation
    )

    results_list.append(results_filtered)

Evaluating detections...
 100% |█████████████████████| 0/0 [155.3ms elapsed, ? remaining, ? samples/s] 
Performing IoU sweep...
 100% |█████████████████████| 0/0 [155.8ms elapsed, ? remaining, ? samples/s] 
Evaluating detections...
 100% |█████████████████| 164/164 [1.5s elapsed, 0s remaining, 89.0 samples/s]          
Performing IoU sweep...
 100% |█████████████████| 164/164 [853.0ms elapsed, 0s remaining, 192.3 samples/s]      
Evaluating detections...
 100% |███████████████| 1224/1224 [16.8s elapsed, 0s remaining, 83.0 samples/s]       
Performing IoU sweep...
 100% |███████████████| 1224/1224 [7.5s elapsed, 0s remaining, 145.5 samples/s]      
Evaluating detections...
 100% |███████████████| 3582/3582 [30.0s elapsed, 0s remaining, 136.4 samples/s]      
Performing IoU sweep...
  25% |███\-----------|  895/3582 [3.9s elapsed, 11.7s remaining, 241.6 samples/s]    

In [ ]:
rows_df = []
# 186**2 is for visualization purposes, representing [156-]
for res, box_area in zip(results_list, boxes_areas + [186**2]):
    res_map = round((max(res.mAP(), 0) * 100), 1)
    res_f1 = round(res.metrics()['fscore'], 3)
    box_size = int(np.sqrt(box_area))
    rows_df.append([box_area, box_size, res_map, res_f1])
    print(f"mAP: {res_map}, F1: {res_f1}")

df_res = pd.DataFrame(data=rows_df, columns=['area', 'box_size', 'map', 'f1'])
print(df_res)

In [ ]:
import plotly.express as px

# mAP
fig = px.line(df_res, x="box_size", y="map", text="map", title=f"mAP per bounding box size for {dataset_orig.name}")
fig.update_traces(textposition="bottom right")

fig.update_xaxes(
    title="Bounding box size"
)
fig.update_yaxes(
    title="mAP"
)

fig.write_image(f"/home/agirbau/work/politics/figures/results_map_face_size_{dataset_orig.name}.pdf")
fig.show()

# F1 score
fig = px.line(df_res, x="box_size", y="f1", text="f1", title=f"F-score per bounding box size for {dataset_orig.name}")
fig.update_traces(textposition="bottom right")

fig.update_xaxes(
    title="Bounding box size"
)
fig.update_yaxes(
    title="F-score"
)

fig.write_image(f"/home/agirbau/work/politics/figures/results_f1_face_size_{dataset_orig.name}.pdf")
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
# F-score
fig.add_trace(
    go.Scatter(x=df_res["box_size"],
            y=df_res["f1"], 
            text=df_res["f1"], 
            name="F-score", 
            mode="lines+markers+text",
            textposition="top left"),
    secondary_y=False,
)
# mAP
fig.add_trace(
    go.Scatter(x=df_res["box_size"],
            y=df_res["map"], 
            text=df_res["map"], 
            name="mAP", 
            mode="lines+markers+text",
            textposition="bottom right"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text=f"F-score and mAP for {dataset_orig.name}"
)

# Set x-axis title
fig.update_xaxes(title_text="Bounding box size x size")

# Set y-axes titles
fig.update_yaxes(title_text="<b>F-score</b>", range=[-0.05, 1.1], secondary_y=False)
fig.update_yaxes(title_text="<b>mAP</b>", range=[-5, 110], secondary_y=True)

fig.write_image(f"/home/agirbau/work/politics/figures/results_f1_map_face_size_{dataset_orig.name}.pdf")
fig.show()

In [30]:
res = results_list[4]
object_methods = [method_name for method_name in dir(res) if callable(getattr(res, method_name))]
print(object_methods)
print(res.metrics())

['__class__', '__delattr__', '__dir__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_confusion_matrix', '_from_dict', '_get_class_index', '_get_iou_thresh_inds', '_parse_classes', '_prepare_serial_dict', 'attributes', 'confusion_matrix', 'copy', 'custom_attributes', 'from_dict', 'from_json', 'from_str', 'get_class_name', 'mAP', 'metrics', 'plot_confusion_matrix', 'plot_pr_curves', 'print_report', 'report', 'serialize', 'to_str', 'write_json']
{'accuracy': 0.9489795918367347, 'precision': 0.9634185820653933, 'recall': 0.9844525305987429, 'fscore': 0.9738219895287957, 'support': 3023}
